In [ ]:

#@title Train LoRA
from google.colab import drive
drive.mount('/content/drive')

!apt -y update -qq
!wget http://launchpadlibrarian.net/367274644/libgoogle-perftools-dev_2.5-2.2ubuntu3_amd64.deb
!wget https://launchpad.net/ubuntu/+source/google-perftools/2.5-2.2ubuntu3/+build/14795286/+files/google-perftools_2.5-2.2ubuntu3_all.deb
!wget https://launchpad.net/ubuntu/+source/google-perftools/2.5-2.2ubuntu3/+build/14795286/+files/libtcmalloc-minimal4_2.5-2.2ubuntu3_amd64.deb
!wget https://launchpad.net/ubuntu/+source/google-perftools/2.5-2.2ubuntu3/+build/14795286/+files/libgoogle-perftools4_2.5-2.2ubuntu3_amd64.deb
!apt install -qq libunwind8-dev
!dpkg -i *.deb
%env LD_PRELOAD=libtcmalloc.so
!rm *.deb

!apt -y install -qq aria2
!pip install torch==1.13.1+cu116 torchvision==0.14.1+cu116 torchaudio==0.13.1 torchtext==0.14.1 torchdata==0.5.1 --extra-index-url https://download.pytorch.org/whl/cu116 -U
!pip install -q xformers==0.0.16 triton==2.0.0 -U

!git clone https://github.com/camenduru/sd-scripts
!sed -i -e 's/requests==2.28.2/# requests==2.28.2/' /content/sd-scripts/requirements.txt
%cd /content/sd-scripts
!pip install -q -r requirements.txt

model_url = "https://civitai.com/api/download/models/95489" #@param {type:"string"}
model_name = "anyloraCheckpoint_bakedvaeBlessedFp16" #@param {type:"string"}
pretrained_model_name_or_path = f"/content/model/{model_name}"

!mkdir /content/model
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M "{model_url}" -d /content/model -o {model_name}

root_path = "/content/drive/MyDrive/MVC_HW" #@param {type:"string"}
instance_prompt = "LoRA_dog" #@param {type:"string"}
images_path = f"{root_path}/{instance_prompt}"
images_tags_json = f"{images_path}/{instance_prompt}.json"
images_latents_json = f"{images_path}/{instance_prompt}-latents.json"
output_dir = f"content/drive/MyDrive/MVC_HW/{instance_prompt}"
max_train_steps = 1600 #@param {type:"integer"}

!python -u finetune/tag_images_by_wd14_tagger.py {images_path} --repo_id SmilingWolf/wd-v1-4-convnext-tagger-v2 --model_dir wd14_tagger_model --thresh 0.35 --batch_size 1 --caption_extension .txt
!python -u finetune/merge_dd_tags_to_metadata.py {images_path} {images_tags_json} --caption_extension .txt
!python -u finetune/prepare_buckets_latents.py {images_path} {images_tags_json} {images_latents_json} {pretrained_model_name_or_path} --batch_size 1 --max_resolution 512,512 --min_bucket_reso 256 --max_bucket_reso 1024 --bucket_reso_steps 64 --mixed_precision no
!python -u train_network.py --pretrained_model_name_or_path {pretrained_model_name_or_path} --train_data_dir {images_path} --in_json {images_latents_json} --output_dir {output_dir} --xformers --max_train_steps {max_train_steps} --use_8bit_adam --network_module networks.lora

!mv {output_dir}/last.safetensors {output_dir}/{instance_prompt}.safetensors